In [53]:
import requests
import json

def getAnnotation():
    url = "http://localhost:4321/api/queries/findAnnotation"
    response = requests.get(url)
    id = json.loads(response.text)[0]
    print(id)
    return id

def writeCutoutGeometryForAnnotationToFile(id, mappedLayers, height, filename):
    url = "http://localhost:4321/api/queries/queryAnnotationAsObj"
    payload = {"annotationId": id, "queryAttributes": [mappedLayers], "distanceToPlane": height}
    print(payload)
    response = requests.post(url, json=payload)
    response.raise_for_status() 
    file = open(filename, "w")
    file.write(response.text)
    file.close()

def getAnnotationGeometry(id, mappedLayers, height):
    url = "http://localhost:4321/api/queries/queryAnnotationAsJson"
    payload = {"annotationId": id, "queryAttributes": mappedLayers, "distanceToPlane": height}
    response = requests.get(url, json=payload)
    if response.status_code == 200:
        return json.loads(response.content)
    else:
        raise Exception("PRo3D request failed.")

In [54]:
id = getAnnotation() # just use the first annotation for simplicity of the demo.

# instead of Ele.aara any other 
# attribute available in the reconstruction can be used
geometry = getAnnotationGeometry(id, ["Ele.aara", "Lat.aara", "Lon.aara"], 100.0) ## all vertices within 100.0 of the surface (in sky direction)


f26c7440-4592-4d5f-abcf-3f720b145590


In [55]:
length = len(geometry["filteredPatches"]) # number of surfaces intersected. will be 1 in most cases
filteredPatches = geometry["filteredPatches"]
patch = filteredPatches[0]
# geometry
vertices = patch["verticesWorldSpace"]
# how to build triangles in geometry array
indices = patch["indices"]

In [57]:
latitudes = patch["Lat.aara"]
longitudes = patch["Lon.aara"]
print('xyz: ' + str(vertices[0]) + ', lat/lon: ' + str(latitudes[0]) + ',' + str(longitudes[0]) + ')')

xyz: [-84.59299553369532, -24.77811020895996, 11.430795891594238], lat/lon: 143.0,139.0)


In [52]:
# now also write the cutout geometry to a object file.
writeCutoutGeometryForAnnotationToFile(id, ["Ele.aara"], "./result.obj")

{'annotationId': 'f26c7440-4592-4d5f-abcf-3f720b145590', 'queryAttributes': [['Ele.aara']], 'distanceToPlane': 100.0}


HTTPError: 400 Client Error: Bad Request for url: http://localhost:4321/api/queries/queryAnnotationAsObj